In [1]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client
import dask.dataframe as dd



In [2]:
cluster = PBSCluster(
    cores=1,
    memory="8GB",
    project='pangeo',
    processes=1,
    walltime='04:00:00',
    local_directory='$TMPDIR')
cluster

PBSCluster('tcp://10.120.43.34:34788', workers=0, threads=0, memory=0 B)

In [3]:
cluster.scale(40)

In [4]:
client = Client(cluster) # scheduler_file="/home/ad/briolf/scheduler.json")
client

Client Scheduler: tcp://10.120.43.34:34788 Dashboard: http://10.120.43.34:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [5]:
import zarr
import netCDF4
import pytide
import xarray as xr
import os

In [6]:
%%time
root = "/work/ALT/odatis/eNATL60/BLBT02/gridT-2D/"
files = [
    os.path.join(root, item) for item in os.listdir(root) if item.endswith(".nc")]

drop_vars = [
    'nav_lat',
    'nav_lon',
    'somxl010',
    'sosaline',
    'sosstsst']

# these are variables I want to drop while running `open_mfdataset` but then add back later
extra_coord_vars = ['time_counter', 'y', 'x']
extra_coord_vars = []

chunks = dict(time_counter=1)

open_kwargs = dict(drop_variables=(drop_vars + extra_coord_vars),
                   chunks=chunks, decode_cf=True, concat_dim="time_counter") #, combine='nested')
ds = xr.open_mfdataset(files, combine='nested',parallel=True, **open_kwargs)


CPU times: user 4.6 s, sys: 240 ms, total: 4.84 s
Wall time: 16.5 s


In [7]:
time=ds.time_counter
ssh=ds.sossheig

In [8]:
time

,Array,Chunk
Bytes,93.50 kB,8 B
Shape,"(11688,)","(1,)"
Count,23863 Tasks,11688 Chunks
Type,datetime64[ns],numpy.ndarray


In [9]:
t=time.values

In [10]:
wt = pytide.WaveTable()
wt = pytide.WaveTable(["M2", "S2", "N2", "O1", "K1"])

In [11]:
%%time
f, vu = wt.compute_nodal_modulations(t)

CPU times: user 13.8 ms, sys: 962 µs, total: 14.7 ms
Wall time: 13.5 ms


In [ ]:
%%time
w = wt.harmonic_analysis(ssh.values, f, vu)

In [ ]:
%%time
hp = wt.tide_from_tide_series(t,w)